# COVID-19 Hospitalization, ICU & Mortality Dashboard Update
Center for Human Dynamics in the Mobile Age (HDMA) at San Diego State University

Jessica Embury

### Modules

In [197]:
import arcgis
from arcgis.gis import GIS
from arcgis import geometry
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer
from arcgis.mapping import WebMap
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import json
import sys
import webbrowser

### User Entered Variables

In [198]:
###############
###SET DATES###
###############
file_date = '26DEC2020'
formatted_date = '122620'

##########################
###SET PATHS IN AND OUT###
##########################
#PATHS IN
sra_in = './data/sra_info.csv'
hosp_in = '../covid_data/hosp_icu_mort/COVID19_HospRate_{}.csv'.format(file_date)
icu_in = '../covid_data/hosp_icu_mort/COVID19_ICURate_{}.csv'.format(file_date)
mort_in = '../covid_data/hosp_icu_mort/COVID19_MortalityRate_{}.csv'.format(file_date)

#PATHS OUT
hosp_out = './data/covid19_hosp_upload.csv'
icu_out = './data/covid_icu_upload.csv'
mort_out = './data/covid_mort_upload.csv'

#Cumulative Dates Files
hosp_dates = './data/covid19_hospitalization.csv'
icu_dates = './data/covid_icu.csv'
mort_dates = './data/covid_mortality.csv'

hosp_db = 'C:/Users/jesse/Dropbox/Mapping-Vulearable-Pop-Tasks/SD-County-Data/COVID-Data-Share-at-HDMA-Center/Hosp-ICU-Mortalities/covid19_hospitalization_{}.csv'.format(formatted_date)
icu_db = 'C:/Users/jesse/Dropbox/Mapping-Vulearable-Pop-Tasks/SD-County-Data/COVID-Data-Share-at-HDMA-Center/Hosp-ICU-Mortalities/covid19_icu_{}.csv'.format(formatted_date)
mort_db = 'C:/Users/jesse/Dropbox/Mapping-Vulearable-Pop-Tasks/SD-County-Data/COVID-Data-Share-at-HDMA-Center/Hosp-ICU-Mortalities/covid19_mortalities_{}.csv'.format(formatted_date)

######################################
###ARCGIS ACCOUNT LOGIN INFORMATION###
######################################
#reference for authentication schemes: https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/
pro = "pro"
#portal =''
#username=''
#password=''

################################
###ARCGIS DETAILS FOR UPDATES###
################################
hosp_layer_id = '2f3443de1a9345d79316484d6c273342'
hosp_map_id = '3b5af1a898ae49e3825bac21badea77c'
hosp_csv_id = '0413f1006e94418b8ecd3af609ec5dff'

icu_layer_id ='164de458148f4c9db3708cc8e99ddbc8'
icu_map_id = '128d6eb1547c4958818a47464da1a079'
icu_csv_id = '7dc2c321393e402b9931d5595fae8678'

mort_layer_id = '1bf213866fad441db609577cad03c7d9'
mort_map_id = '64f43b6d62ef46c18bf6db55fce5afce'
mort_csv_id = '11047e66ca6f483d8873078ef392e18c'

dash = 'https://arcg.is/0amqvz'

### Connect to ArcGIS Account

In [199]:
gis = GIS(pro)
#gis = GIS(portal, username, password)

### Functions for Map Modifications

In [200]:
def get_map (map_id):
    '''
    GET MAP DATA FOR SYMBOLOGY CHANGES
    '''    
    m = gis.content.get(map_id)
    data = m.get_data()
    print(m)    
    return data

def update_map (map_id, data):
    '''
    UPDATE MAP TO SAVE CHANGES
    '''
    m = gis.content.get(map_id)  
    # Set the item_properties to include the desired update
    properties = {"text": json.dumps(data)}
    # 'Commit' the updates to the Item
    update = m.update(item_properties=properties)    
    return update

# HOSPITALIZATION

### Format Data

In [201]:
#BASE SRA DATA
sra = pd.read_csv(sra_in)
print(len(sra))
sra.head()

41


,sra_num,sra_name,latitude,longitude
0,1,Central San Diego,32.722644,-117.141073
1,2,Peninsula,32.742274,-117.216035
2,3,Coronado,32.657317,-117.143062
3,4,National City,32.665847,-117.099737
4,5,Southeastern San Diego,32.685705,-117.038621


In [202]:
hosp = pd.read_csv(hosp_in)
hosp = hosp.rename(columns = {'date':'Date'})
date = hosp['Date'][0]
print(date)
hosp.head()

12/26/2020


,Geography,Allocated Hospitalization Count (Raw),COVID-19 Total Hospitalization Rate (Suppressed),Date
0,Alpine,20.8,129.0,12/26/2020
1,Anza-Borrego Springs,NaN,NaN,12/26/2020
2,Carlsbad,76.0,60.1,12/26/2020
3,Central Region,1138.0,220.9,12/26/2020
4,Central San Diego,365.5,173.3,12/26/2020


In [204]:
del hosp['COVID-19 Total Hospitalization Rate (Suppressed)']
hosp = hosp.rename(columns = {'Allocated Hospitalization Count (Raw)':'hosp_count', 'Geography':'sra_name'})
hosp = hosp.fillna(0)
hosp.head()

,sra_name,hosp_count,Date
0,Alpine,20.8,12/26/2020
1,Anza-Borrego Springs,0.0,12/26/2020
2,Carlsbad,76.0,12/26/2020
3,Central Region,1138.0,12/26/2020
4,Central San Diego,365.5,12/26/2020


In [205]:
#merge to sra df with coordinates
hosp_merge = sra.merge(hosp, how='left', on='sra_name')
hosp_merge['label'] = hosp_merge['hosp_count'].astype(str)
print(len(hosp_merge))
hosp_merge.head()

41


,sra_num,sra_name,latitude,longitude,hosp_count,Date,label
0,1,Central San Diego,32.722644,-117.141073,365.5,12/26/2020,365.5
1,2,Peninsula,32.742274,-117.216035,30.0,12/26/2020,30.0
2,3,Coronado,32.657317,-117.143062,11.0,12/26/2020,11.0
3,4,National City,32.665847,-117.099737,221.1,12/26/2020,221.1
4,5,Southeastern San Diego,32.685705,-117.038621,468.0,12/26/2020,468.0


In [206]:
#export csv for feature layer overwrite
hosp_merge.to_csv(hosp_out, index=False)

In [207]:
#save csv with a column for every date
#prepare new data for addition to existing csv
hosp_temp = hosp_merge[['sra_name','hosp_count']]
hosp_temp = hosp_temp.rename(columns = {'hosp_count':'{}'.format(date)})
hosp_temp.head()

,sra_name,12/26/2020
0,Central San Diego,365.5
1,Peninsula,30.0
2,Coronado,11.0
3,National City,221.1
4,Southeastern San Diego,468.0


In [208]:
#add new date column to existing csv
hosp_cum = pd.read_csv(hosp_dates)
hosp_cum = hosp_cum.merge(hosp_temp, on='sra_name')
hosp_cum.to_csv(hosp_dates, index=False)
hosp_cum.to_csv(hosp_db, index=False)
hosp_cum.head()

,sra_num,sra_name,latitude,longitude,6/3/2020,6/10/2020,6/17/2020,6/24/2020,7/1/2020,7/8/2020,7/14/2020,7/28/2020,8/4/2020,8/11/2020,8/18/2020,8/24/2020,8/29/2020,9/5/2020,9/19/2020,9/26/2020,10/3/2020,10/10/2020,10/17/2020,10/24/2020,10/31/2020,11/7/2020,11/14/2020,11/21/2020,11/28/2020,12/12/2020,12/26/2020
0,1,Central San Diego,32.722644,-117.141073,94.39,102.22,109.7,114.9,127.6,138.8,150.6,176.5,189.0,195.0,198.1,199.9,205.7,207.7,217.5,225.6,238.9,242.1,248.0,250.8,262.4,269.5,282.0,293.2,304.1,335.2,365.5
1,2,Peninsula,32.742274,-117.216035,7.67,7.67,9.7,9.7,10.6,10.6,12.3,14.8,14.8,16.2,16.7,17.7,17.7,18.7,18.7,18.7,18.7,18.7,18.7,18.7,19.7,21.1,22.6,23.6,24.0,26.9,30.0
2,3,Coronado,32.657317,-117.143062,0.00,0.00,0.0,5.0,5.0,5.0,5.0,6.0,7.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,9.0,9.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,11.0,11.0
3,4,National City,32.665847,-117.099737,81.06,82.99,88.8,97.5,103.3,110.0,117.7,124.6,131.3,137.1,139.0,141.9,145.8,150.6,156.4,163.2,164.1,166.1,167.0,169.0,173.8,177.7,180.6,186.4,193.1,205.7,221.1
4,5,Southeastern San Diego,32.685705,-117.038621,118.17,125.60,141.4,150.3,168.2,184.6,200.2,245.2,263.0,274.8,284.4,289.9,294.9,304.7,312.7,320.6,323.6,333.5,341.0,346.9,352.3,356.7,362.3,374.9,388.6,429.6,468.0


### Overwrite Hospitalization Feature Layer and CSV File

In [209]:
#overwrite master csv with all dates data
csv_item = gis.content.get(hosp_csv_id)
csv_item.update({}, hosp_dates)

True

In [210]:
#get feature layer containing updated data 
hosp_layer = gis.content.get(hosp_layer_id)
hosp_layer

hosp_layer_collection = FeatureLayerCollection.fromitem(hosp_layer)

#call the overwrite() method which can be accessed using the manager property
hosp_layer_collection.manager.overwrite(hosp_out)

{'success': True}

### Modify Hospitalization Map Symbology

In [211]:
#get hospitalization max for graduated point symbology
hosp_max = hosp_merge['hosp_count'].max()

#get map data
hosp_map = get_map(hosp_map_id)

#set max value for graduated points symbols
hosp_map['operationalLayers'][1]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = hosp_max.item()
hosp_map['operationalLayers'][1]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = hosp_max.item()
hosp_map['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = hosp_max.item()
hosp_map['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = hosp_max.item()

#update map to save changes
hosp_map_update = update_map(hosp_map_id, hosp_map)
hosp_map_update

<Item title:"COVID-19 Hospitalizations by San Diego SRA" type:Web Map owner:jembury8568_SDSUGeo>


True

# MORTALITY

### Format Data

In [212]:
mort = pd.read_csv(mort_in)
mort = mort.rename(columns = {'date':'Date'})
date = mort['Date'][0]
print(date)

mort.head()

12/26/2020


,Geography,Allocated Death Count (Raw),COVID-19 Total Mortality Rate (Suppressed),Date
0,Alpine,5.4,33.7,12/26/2020
1,Anza-Borrego Springs,NaN,NaN,12/26/2020
2,Carlsbad,15.0,11.8,12/26/2020
3,Central Region,251.0,48.7,12/26/2020
4,Central San Diego,87.4,41.5,12/26/2020


In [214]:
del mort['COVID-19 Total Mortality Rate (Suppressed)']
mort = mort.rename(columns = {'Allocated Death Count (Raw)':'mort_count', 'Geography':'sra_name'})
mort = mort.fillna(0)
mort.head()

,sra_name,mort_count,Date
0,Alpine,5.4,12/26/2020
1,Anza-Borrego Springs,0.0,12/26/2020
2,Carlsbad,15.0,12/26/2020
3,Central Region,251.0,12/26/2020
4,Central San Diego,87.4,12/26/2020


In [215]:
#merge to sra df with coordinates
mort_merge = sra.merge(mort, how='left', on='sra_name')
mort_merge['label'] = mort_merge['mort_count'].astype(str)
print(len(mort_merge))
mort_merge.head()

41


,sra_num,sra_name,latitude,longitude,mort_count,Date,label
0,1,Central San Diego,32.722644,-117.141073,87.4,12/26/2020,87.4
1,2,Peninsula,32.742274,-117.216035,7.2,12/26/2020,7.2
2,3,Coronado,32.657317,-117.143062,5.0,12/26/2020,5.0
3,4,National City,32.665847,-117.099737,71.4,12/26/2020,71.4
4,5,Southeastern San Diego,32.685705,-117.038621,115.8,12/26/2020,115.8


In [216]:
#export csv for feature layer overwrite
mort_merge.to_csv(mort_out, index=False)

In [217]:
#save csv with a column for every date
#prepare new data for addition to existing csv
mort_temp = mort_merge[['sra_name','mort_count']]
mort_temp = mort_temp.rename(columns = {'mort_count':'{}'.format(date)})
mort_temp.head()

,sra_name,12/26/2020
0,Central San Diego,87.4
1,Peninsula,7.2
2,Coronado,5.0
3,National City,71.4
4,Southeastern San Diego,115.8


In [218]:
#add new date column to existing csv
mort_cum = pd.read_csv(mort_dates)
mort_cum = mort_cum.merge(mort_temp, on='sra_name')
mort_cum.to_csv(mort_dates, index=False)
mort_cum.to_csv(mort_db, index=False)
mort_cum.head()

,sra_num,sra_name,latitude,longitude,6/3/20,6/10/2020,6/17/2020,6/24/2020,7/1/2020,7/8/2020,7/14/2020,7/28/2020,8/4/2020,8/11/2020,8/18/2020,8/24/2020,8/29/2020,9/5/2020,9/19/2020,9/26/2020,10/3/2020,10/10/2020,10/17/2020,10/24/2020,10/31/2020,11/7/2020,11/14/20,11/21/20,11/28/20,12/12/2020,12/26/2020
0,1,Central San Diego,32.722644,-117.141073,15.584,16.985,19.1,21.8,23.5,25.6,28.8,35.8,38.5,40.5,40.9,41.3,41.3,42.3,43.6,44.2,50.3,50.7,52.4,53.8,57.2,59.5,64.1,67.1,69.3,76.7,87.4
1,2,Peninsula,32.742274,-117.216035,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,5.2,5.2,5.2,5.2,5.7,5.7,5.7,5.7,5.7,5.7,5.7,5.7,5.7,6.1,6.1,6.1,6.1,6.1,7.2
2,3,Coronado,32.657317,-117.143062,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0
3,4,National City,32.665847,-117.099737,21.224,23.154,24.1,25.1,26.0,32.8,34.7,38.6,41.5,41.5,43.4,43.4,43.4,46.3,48.3,48.3,50.2,51.2,52.1,52.1,53.1,53.1,54.1,55.0,56.0,59.9,71.4
4,5,Southeastern San Diego,32.685705,-117.038621,16.742,17.457,19.4,20.3,24.4,32.1,41.5,63.4,64.7,66.8,70.3,73.0,73.3,77.2,81.1,82.7,83.4,84.4,85.5,88.2,89.1,89.1,89.4,94.2,95.8,104.9,115.8


### Overwrite Mortality Feature Layer and CSV File

In [219]:
#overwrite master csv with all dates data
csv_item = gis.content.get(mort_csv_id)
csv_item.update({}, mort_dates)

True

In [220]:
#get feature layer containing updated data 
mort_layer = gis.content.get(mort_layer_id)
mort_layer

mort_layer_collection = FeatureLayerCollection.fromitem(mort_layer)

#call the overwrite() method which can be accessed using the manager property
mort_layer_collection.manager.overwrite(mort_out)

{'success': True}

### Modify Mortality Map Symbology

In [221]:
#get mortality max for graduated point symbology
mort_max = mort_merge['mort_count'].max()

#get map data
mort_map = get_map(mort_map_id)

#set max value for graduated points symbols
mort_map['operationalLayers'][1]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = mort_max.item()
mort_map['operationalLayers'][1]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = mort_max.item()
mort_map['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = mort_max.item()
mort_map['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = mort_max.item()

#update map to save changes
mort_map_update = update_map(mort_map_id, mort_map)
mort_map_update

<Item title:"COVID-19 Mortality by San Diego SRA" type:Web Map owner:jembury8568_SDSUGeo>


True

In [222]:
webbrowser.open(dash, new=2)

True